### Домашнее задание 1.

Автор: Владимир Лозовой.

Задачи:
1. Функция для NPV
2. Функция для IRR
3. Функция для перевода из compounding начисления в непрерывное и обратно, на всякий случай

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from typing import Union

In [29]:
# на всякий случай написал функцию для конвертации между разными типами начисления
def comp_to_cont(r:Union[int, float],
                 m:int,
                 direction:str='direct'):
    """Converts compounding annual rate to continious and vice versa"""
    if direction=='direct':
        return m * np.log(1+r/m)
    return m * (np.exp(r / m) - 1)
        
    
def npv(price: Union[int, float],
        face_value: Union[int, float],
        coupon: Union[int, float],
        frequency: int,
        T:int,
        r:Union[int, float],
        rate_type:str='compounding') -> float:
    """
    Returns NPV of a bond or project.
    
    Parameters:
    -----------
    price: int or float.
        Price of bond or initial investment into the project.
    face_value: int or float.
        Face value of bond payed to bondholder at the maturity.
    coupon: int or float.
        Coupon interest in %. If int, will be divided by 100.
    frequency: int.
        Coupon frequency, times per year.
    T: int.
        Years remaining to maturity of the bond.
    r: int or float.
        Annual discount rate for cash flows. If int, will be divided by 100.
    rate_type: str.
        Rate type: 'compounding' or 'continious'.
    """
    if isinstance(coupon, int):
        coupon = coupon/100
    if isinstance(r, int):
        r = r/100
    
    time_factors = np.arange(1/frequency, T+1/frequency, 1/frequency)
    
    if rate_type=='compounding':
        # посчитаем дисконт-факторы    
        discount_factors = np.array([1/(1 + r)**x for x in time_factors]) 
    elif rate_type=='continious':
        discount_factors = np.array([np.exp(-x*r) for x in time_factors])
        
    coupons_pv = np.array([face_value * coupon * x for x in discount_factors])
    face_pv = face_value * discount_factors[-1]
    return np.add(coupons_pv.sum(), face_pv-price)
    

def irr(price: Union[int, float],
        face_value: Union[int, float],
        coupon: Union[int, float],
        frequency: int,
        T:int,
        rate_type:str='compounding') -> float:
    """
    Returns IRR of a bond or project.
    
    Parameters:
    -----------
    price: int or float.
        Price of bond or initial investment into the project.
    face_value: int or float.
        Face value of bond payed to bondholder at the maturity.
    coupon: int or float.
        Coupon interest in %. If int, will be divided by 100.
    frequency: int.
        Coupon frequency, times per year.
    T: int.
        Years remaining to maturity of the bond.
    rate_type: str.
        Rate type: 'compounding' or 'continious'.
    """
    f = lambda x: npv(price, 
                      face_value,
                      coupon,
                      frequency,
                      T,
                      x,
                      rate_type)
    r = scipy.optimize.root(f, [0])
    return r.x[0]

Проверим, что все работает.

In [31]:
npv(price=100, 
    face_value=110, 
    coupon=0, 
    frequency=1, 
    T=1, 
    r=10, 
    rate_type='compounding')

# теперь посчитаем какой-нибудь бонд посложнее в экселе и посмотрим
npv(price=860, 
    face_value=1000, 
    coupon=0.01, 
    # платежи 3 раза в год
    frequency=3, 
    T=2, 
    r=0.1, 
    rate_type='continious')

irr(price=880.2112, 
    face_value=1000, 
    coupon=0.01, 
    frequency=3, 
    T=2, 
    rate_type='compounding')

0.0

12.210216083593096

0.0999999880111052

Все ответы сошлись, все окей.